In [2]:
##metaheuristique
import numpy as np
import requests


In [3]:

url = "https://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/CCNFP10g1a.txt"

try:
    response = requests.get(url)
    response.raise_for_status()  # Vérifie qu'il n'y a pas d'erreur HTTP
    contenu_texte = response.text
    
    # Affiche les premières lignes du fichier
    lignes = contenu_texte.splitlines()
    for i, ligne in enumerate(lignes[:10]):  # Par exemple, afficher 10 lignes
        print(f"Ligne {i+1}: {ligne}")

except requests.exceptions.RequestException as e:
    print(f"Erreur lors de l'ouverture de l'URL : {e}")

Ligne 1:           10
Ligne 2:            2
Ligne 3:            3
Ligne 4:            1
Ligne 5:            3
Ligne 6:            5
Ligne 7:            1
Ligne 8:            4
Ligne 9:            4
Ligne 10:            1


In [4]:
base_url = "https://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/CCNFP10g"


for i in range(10):
    url = base_url + str(i+1) + ".txt"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Vérifie qu'il n'y a pas d'erreur HTTP
        contenu_texte = response.text

        with open("CCNFP10g" + str(i+1) + ".txt", "w") as fichier:
            fichier.write(contenu_texte)


    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de l'ouverture de l'URL : {e}")
    
    